In [17]:
import numpy as np
import pandas as pd
import mne
import os

In [18]:
root_path='/mnt/ssd_4tb_0/data/tuhab_preprocessed'
csv_path='/mnt/ssd_4tb_0/data/tuhab_preprocessed/file_lengths_map.csv'

In [19]:
ex_data_filename = os.listdir(root_path)[0]
ex_data_filename

'aaaaalgi_s001_t001_preprocessed.npy'

In [20]:
ex_data = np.load(os.path.join(root_path, ex_data_filename))
ex_data.shape

(86300, 19)

In [21]:
# from feature_calc import 

In [47]:
import numpy as np
import mne

data = ex_data.T
sfreq = 100.
ch_names = [f'EEG {i+1}' for i in range(data.shape[0])]
ch_types = ['eeg'] * len(ch_names)

info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
raw = mne.io.RawArray(data, info)


Creating RawArray with float64 data, n_channels=19, n_times=86300
    Range : 0 ... 86299 =      0.000 ...   862.990 secs
Ready.


In [48]:
epochs = mne.make_fixed_length_epochs(raw, duration=10.0, preload=True)

Not setting metadata
86 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 86 events and 1000 original time points ...
0 bad epochs dropped


In [49]:
psds, freqs = epochs.compute_psd(fmin=0.5, fmax=35.0, method='multitaper').get_data(return_freqs=True)

    Using multitaper spectrum estimation with 7 DPSS windows


In [52]:
np.sum(psds, axis=-1, keepdims=True)[0]

array([[212.43681443],
       [225.12978775],
       [309.67715001],
       [385.35276375],
       [192.8760953 ],
       [298.36727296],
       [234.36994622],
       [508.73171458],
       [584.111449  ],
       [408.39562704],
       [413.69438719],
       [426.82191434],
       [438.95847778],
       [616.63894498],
       [346.74105505],
       [359.28781286],
       [491.2160006 ],
       [287.50770009],
       [402.6969012 ]])

In [53]:
psds_normalized = psds / np.sum(psds, axis=-1, keepdims=True)  # (n_epochs, n_channels, n_freqs)

In [45]:
alpha_idx = np.logical_and(freqs >= 0.5, freqs <= 4)
# alpha_idx = np.logical_and(freqs >= 8, freqs <= 13)
alpha_power = psds_normalized[:, :, alpha_idx].mean(axis=-1)  # (n_epochs, n_channels)

In [41]:
X = alpha_power.reshape(len(alpha_power), -1)  # shape (n_epochs, n_channels)

In [44]:
X[0]

array([0.026179  , 0.026499  , 0.0257482 , 0.02511483, 0.02481881,
       0.02608465, 0.02617482, 0.02655482, 0.0252386 , 0.01881953,
       0.0262003 , 0.02534393, 0.02553713, 0.02627932, 0.02302163,
       0.02558391, 0.02508292, 0.02476044, 0.02557104])

In [46]:
alpha_power[0]

array([0.026179  , 0.026499  , 0.0257482 , 0.02511483, 0.02481881,
       0.02608465, 0.02617482, 0.02655482, 0.0252386 , 0.01881953,
       0.0262003 , 0.02534393, 0.02553713, 0.02627932, 0.02302163,
       0.02558391, 0.02508292, 0.02476044, 0.02557104])

In [16]:
psds[0, 0]

array([9.95423509e-02, 1.07891592e-01, 9.38131566e-02, 7.76091323e-02,
       6.93776109e-02, 3.71003397e-02, 5.66822224e-02, 5.33265077e-02,
       4.99982869e-02, 3.49185798e-02, 3.20052063e-02, 2.67911287e-02,
       2.62542858e-02, 1.59386322e-02, 1.61030332e-02, 1.52357331e-02,
       1.65196824e-02, 1.55729666e-02, 1.19253644e-02, 8.83588745e-03,
       1.11281095e-02, 1.00509505e-02, 8.34583317e-03, 5.53532303e-03,
       6.87604410e-03, 6.11751983e-03, 5.07796501e-03, 3.43762745e-03,
       3.11694818e-03, 2.61619985e-03, 2.65273425e-03, 2.13252555e-03,
       2.21663728e-03, 2.43530868e-03, 2.43646534e-03, 2.82612679e-03,
       2.62840452e-03, 3.12694584e-03, 2.23770955e-03, 1.95368243e-03,
       1.86517438e-03, 1.67046349e-03, 1.61735101e-03, 1.65847165e-03,
       1.19109669e-03, 8.86006513e-04, 8.38804590e-04, 1.31260209e-03,
       1.44143442e-03, 1.25411717e-03, 1.17690318e-03, 9.40651355e-04,
       9.19704669e-04, 8.47243729e-04, 4.99092289e-04, 4.86886500e-04,
      

In [27]:
freqs

array([ 0.5,  0.6,  0.7,  0.8,  0.9,  1. ,  1.1,  1.2,  1.3,  1.4,  1.5,
        1.6,  1.7,  1.8,  1.9,  2. ,  2.1,  2.2,  2.3,  2.4,  2.5,  2.6,
        2.7,  2.8,  2.9,  3. ,  3.1,  3.2,  3.3,  3.4,  3.5,  3.6,  3.7,
        3.8,  3.9,  4. ,  4.1,  4.2,  4.3,  4.4,  4.5,  4.6,  4.7,  4.8,
        4.9,  5. ,  5.1,  5.2,  5.3,  5.4,  5.5,  5.6,  5.7,  5.8,  5.9,
        6. ,  6.1,  6.2,  6.3,  6.4,  6.5,  6.6,  6.7,  6.8,  6.9,  7. ,
        7.1,  7.2,  7.3,  7.4,  7.5,  7.6,  7.7,  7.8,  7.9,  8. ,  8.1,
        8.2,  8.3,  8.4,  8.5,  8.6,  8.7,  8.8,  8.9,  9. ,  9.1,  9.2,
        9.3,  9.4,  9.5,  9.6,  9.7,  9.8,  9.9, 10. , 10.1, 10.2, 10.3,
       10.4, 10.5, 10.6, 10.7, 10.8, 10.9, 11. , 11.1, 11.2, 11.3, 11.4,
       11.5, 11.6, 11.7, 11.8, 11.9, 12. , 12.1, 12.2, 12.3, 12.4, 12.5,
       12.6, 12.7, 12.8, 12.9, 13. , 13.1, 13.2, 13.3, 13.4, 13.5, 13.6,
       13.7, 13.8, 13.9, 14. , 14.1, 14.2, 14.3, 14.4, 14.5, 14.6, 14.7,
       14.8, 14.9, 15. , 15.1, 15.2, 15.3, 15.4, 15

In [55]:
delta_idx = np.logical_and(freqs >= 0.5, freqs <= 4)
delta_power = psds[:, :, delta_idx].mean(axis=-1)

In [56]:
delta_power[0]

array([ 5.56138347,  5.9657138 ,  7.97362826,  9.67806918,  4.78695476,
        7.78280524,  6.13459003, 13.50927972, 14.74215518,  7.68581327,
       10.83891896, 10.81734436, 11.2097378 , 16.20485224,  7.98254513,
        9.19198631, 12.32113194,  7.11881853, 10.29737928])

In [43]:
np.sum(psds, axis=-1, keepdims=True).shape

(86, 19, 1)

In [54]:
psds, freqs = epochs.compute_psd(fmin=0.5, fmax=35.0, method='multitaper').get_data(return_freqs=True)

    Using multitaper spectrum estimation with 7 DPSS windows


In [49]:
psds[:, :, delta_idx].shape

(86, 19, 336)

In [47]:
psds[:, :, delta_idx].mean(axis=-1).shape

(86, 19)

In [57]:
alpha_idx = np.logical_and(freqs >= 8, freqs <= 13)
alpha_power = psds[:, :, alpha_idx].mean(axis=-1)  # (n_epochs, n_channels)

In [58]:
alpha_power[0]

array([0.04849767, 0.04580473, 0.07337097, 0.08333952, 0.08583394,
       0.08023816, 0.04232253, 0.07630774, 0.14929297, 0.34016601,
       0.10961727, 0.12121456, 0.11708033, 0.11903496, 0.20682328,
       0.14509184, 0.12262967, 0.10910336, 0.08541131])